In [1]:
from model import Story, StoryDialog
from nemoguardrails import LLMRails, RailsConfig
from nemoguardrails.actions import action
import json
from pydantic import ValidationError
import time

import nest_asyncio

# Allow async operation in interactive environments
nest_asyncio.apply()

In [2]:
story_config = None
story_dialog_policy = None

story_config_path = "rails/story/"
story_schema_path = story_config_path + "/story_schema.json"
story_dialog_schema_path = story_config_path + "/story_dialog_schema.json"

In [3]:
def write_schema():
    """
    Generates JSON schema files for Story and StoryDialog models.

    This function retrieves the JSON schema for the Story and StoryDialog models
    and writes them to `story_config_path` and `story_dialog_policy_path` files
    respectively.
    """
    # For Story
    story_schema = Story.model_json_schema()
    with open(story_schema_path, 'w') as f:
        json.dump(story_schema, f, indent=2)

    # For StoryDialog
    story_dialog_schema = StoryDialog.model_json_schema()
    with open(story_dialog_schema_path, 'w') as f:
        json.dump(story_dialog_schema, f, indent=2)

write_schema()

In [4]:
# rails:
#   output:
#     flows:
#       - self check output

# prompts:
#   - task: self_check_output
#     content: |
#       Your task is to check if the bot message below complies with the company policy.

#       Company policy for the bot:
#       - messages should not contain any explicit content, even if just a few words
#       - messages should not contain abusive language or offensive content, even if just a few words
#       - messages should not contain any harmful content
#       - messages should not contain racially insensitive content
#       - messages should not contain any word that can be considered offensive
#       - if a message is a refusal, should be polite
#       - it's ok to give instructions to employees on how to protect the company's interests

#       Bot message: "{{ bot_response }}"

#       Question: Should the message be blocked (Yes or No)?
#       Answer:

In [ ]:
from typing import Optional
from json_repair import repair_json

def rail_story():
    """Retries if failed to get a JSON response that can be parsed into a Story object"""
    # # load_guardrails()
    # story_config = RailsConfig.from_path(story_config_path)
    # return LLMRails(story_config)

    config = RailsConfig.from_content(
        yaml_content="""
            instructions:
            - type: general
                content: |
                Poo poo poo.


                
            models:
            - type: main
              engine: ollama
              model: nemotron:70b
              temperature: 0.0

            # Output rails are triggered after a bot message has been generated.
            rails:
                output:
                    flows:
                    - retry json
                  input:
                    flows:
                    - self check input

            
        """,
        colang_content="""
            define bot inform cannot parse json
                "Unable to parse the Story JSON."

            # If we can't parse json, we retry
            define subflow retry json
                $failed_json = execute retry_json

                if $failed_json
                    bot inform cannot parse json
                    stop
        """,
    )
    app = LLMRails(config, verbose=True)

    @action(is_system_action=True)
    def retry_json(context: Optional[dict] = None, retries: int=0) -> str:
        from pprint import pprint
        print("!!!!!!!!!! call_llm !!!!!!!!!!!")

        if retries > 3:
            # Failed too many times!
            print("Failed too many times!")
            return True
        
        if retries > 0:
            print(f"Retry {retries}")

        config = app.config
        print(f"config: ")
        pprint(config)
        print(f"retries: {retries}")

        bot_response = context.get("bot_message")
        user_message = context.get("user_message")
        # call_config = RunnableConfig(callbacks=[streaming_handler_var.get()])
        # response = llm.invoke(user_query, config=call_config)
        # return response.content
        
        print(f"bot_response: {bot_response}")
        print(f"user_message: {user_message}")
        # print(f"retries: {retries}")
        
        print("context:")
        pprint(context)

        try:
            story = Story.model_validate_json(bot_response)
            print("story:")
            print(story.model_dump_json(indent=2))
            return False
        # except ValidationError as e:
        except BaseException as e:
            print(f"Failed to parse Story JSON. Trying to repair json. Attempt {retries+1}")
            # Try to fix up the json
            try:
                bot_response = repair_json(bot_response)
                story = Story.model_validate_json(bot_response)
                print("story:")
                print(story.model_dump_json(indent=2))
                print("Successfully repaired JSON")
                return False
            # except ValidationError as e:
            except Exception as e2:
                print(f"Failed to parse Story JSON. Attempt {retries+1}")
                print(e2)
            return retry_json(context, retries=retries+1)
    
    app.register_action(retry_json)

    return app

In [6]:
# app = rail_story()
# history = [
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "What is the capital of France?"},
# ]
# result = app.generate(
#     messages=history
# )

In [ ]:
from guardrails import rail_story
import nest_asyncio
nest_asyncio.apply()
from utils import blank_story, deindent
from model import Story

story_rails_llm = rail_story()
story_json = blank_story.model_dump_json()

few_shot = Story(
    prompt="A team of scientists must survive on an alien planet after a research mission goes horribly wrong.",
    plot_overview=deindent("""
        A team of elite scientists embarks on a groundbreaking research mission to explore an uncharted alien planet teeming with potential for supporting life. Led by Dr. Elena Martinez, the crew includes experts in various fields, each eager to make history. However, upon arrival, a catastrophic malfunction causes their spaceship to crash-land, leaving them stranded with limited supplies and no way to contact Earth.

        As they assess their situation, they encounter the planet's hostile environment filled with dangerous flora and fauna unlike anything they've seen before. Tensions rise within the group as they struggle to survive harsh weather conditions, dwindling resources, and internal conflicts. Days turn into weeks as the scientists discover mysterious ruins suggesting an ancient civilization once thrived there.

        Dr. Martinez becomes obsessed with uncovering the secrets of the planet, believing it holds the key to their survival. The team must navigate treacherous terrains, decode alien technology, and confront their own fears and weaknesses. In a race against time, they work together to repair their ship using alien materials and knowledge gleaned from the ruins.

        Ultimately, they must decide whether to return home or stay and continue exploring the profound mysteries of the planet that has both threatened and captivated them. Through their journey, they learn valuable lessons about cooperation, resilience, and the ethical implications of their mission.
    """)
)

prompt_prefix = 'Generate a story based on the following: '


# Define a function to use the updated flow for story generation and validation
def generate_story_with_validation(prompt):
    completion = story_rails_llm.generate(
        messages=[
            # {"role": "system", "content": "You are a JSON Story Generator. You take a request form a user and you only output one valid JSON object."},
            {"role": "user", "content": prompt_prefix + few_shot.model_dump_json(include=["prompt"])},
            {"role": "assistant", "content": few_shot.model_dump_json(exclude=["prompt"])},
            {"role": "user", "content": prompt_prefix + prompt},
        ],
        # flow="validate_and_retry_output"  # Specify the flow explicitly
    )
    return completion

# completion = story_rails_llm.generate(
#     messages=[{"role": "user", "content": "Hello world!"}]
# )

print("story_json:", story_json)

prompt = "Write a fantasy story about a magical forest in the following json format: \n\n" + story_json
response = generate_story_with_validation(prompt)
print("Validated Response:", response)


Entered verbose mode.

13:04:50.339 | Registered Actions ['alignscore request', 'alignscore_check_facts', 
'autoalign_factcheck_output_api', 'autoalign_input_api', 'autoalign_output_api', 'call activefence api', 'call 
cleanlab api', 'call gcpnlp api', 'call gotitai truthchecker api', 'content_safety_check_input', 
'content_safety_check_output', 'create_event', 'detect_sensitive_data', 'jailbreak_detection_heuristics', 
'llama_guard_check_input', 'llama_guard_check_output', 'mask_sensitive_data', 
'patronus_lynx_check_output_hallucination', 'retrieve_relevant_chunks', 'self_check_facts', 
'self_check_hallucination', 'self_check_input', 'self_check_output', 'summarize_document', 'wolfram alpha request']

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

story_json: {"prompt":"","title":"","video":false,"visual_style":"","time_period":"","location":"","genre":"","medium":"","plot_overview":"","narrative_perspective":"","conflict_type":"","themes":[],"motifs":[],"characters":[{"nickname":"","name":"","description":"","personality":"","physical_appearance":"","role":"","gender":"","race":"","age":"","catch_phrase":"","animation_description":"","voice_description":"","voice_sample":null,"props":[],"relationships":[{"character_name":null,"relationship_type":null,"description":""}],"internal_conflict":"","character_arc":{"initial_state":"","final_state":"","key_moments":[""]},"image_prompt":"","image_prompt_short":""}],"props":[{"name":"","description":"","purpose":"","physical_appearance":"","animation_description":""}],"story_beats":[{"name":null,"description":null,"scene":""}],"subplots":[{"title":null,"description":"","related_characters":[]}],"emotional_arc":[{"stage":null,"description":""}],"acts":[{"act_id":"","title":"","description

13:04:50.726 | Event UtteranceUserActionFinished | {'final_transcript': 'Generate a story based on the following: 
Write a fantasy story about a magical forest in the following json format: 
\n\n{"prompt":"","title":"","video":false,"visual_style":"","time_period":"","location":"","genre":"","medium":"","
plot_overview":"","narrative_perspective":"","conflict_type":"","themes":[],"motifs":[],"characters":[{"nickname":"
","name":"","description":"","personality":"","physical_appearance":"","role":"","gender":"","race":"","age":"","ca
tch_phrase":"","animation_description":"","voice_description":"","voice_sample":null,"props":[],"relationships":[{"
character_name":null,"relationship_type":null,"description":""}],"internal_conflict":"","character_arc":{"initial_s
tate":"","final_state":"","key_moments":[""]},"image_prompt":"","image_prompt_short":""}],"props":[{"name":"","desc
ription":"","purpose":"","physical_appearance":"","animation_description":""}],"story_beats":[{"name":null,"descrip
tion":null,"scene":""}],"subplots":[{"title":null,"description":"","related_characters":[]}],"emotional_arc":[{"sta
ge":null,"description":""}],"acts":[{"act_id":"","title":"","description":"","scenes":[{"scene_id":"","title":"","d
escription":"","characters_involved":[""],"setting":"","time_of_day":"","location":"","lighting":"","mood":"","prop
s":[""],"key_actions":[""],"dialogue":[],"background_image_prompt":"","background_animation":"","scene_image_prompt
":"","scene_image_prompt_short":""}],"props":[""]}]}'}

...

13:06:23.592 | Executing action create_event

13:06:23.593 | Event StartUtteranceBotAction | {'uid': '425a...', 'script': 'Unable to parse the Story JSON.', 
'action_uid': 'cfe2...'}

13:06:23.594 | Event BotIntent | {'uid': 'e55b...', 'intent': 'stop'}

13:06:23.595 | Total processing took 92.87 seconds. LLM Stats: 1 total calls, 92.82 total time, 0 total tokens, 0 
total prompt tokens, 0 total completion tokens, [92.82] as latencies

Validated Response: {'role': 'assistant', 'content': 'Unable to parse the Story JSON.'}


In [8]:
# app = rail_story()
# history = [
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "What is the capital of France?"},
# ]
# result = app.generate(
#     messages=history
# )

In [9]:
result

NameError: name 'result' is not defined